# Cleaning Data: Homework

In class, we discussed a few different ways to clean data — simple Python methods, Pandas versions of those methods, and regular expressions (both in plain Python and in Pandas). You are welcome to use whatever combination of those approaches you'd like. If you really want to stretch your skills, try solving the questions a few different ways.

## 0) Load `pets.csv` (provided in CourseWorks)

... and assign the data to a variable named `pets`.

In [74]:
import pandas as pd
import requests
import numpy as np

In [4]:
pets = pd.read_csv('pets.csv')
pets

,name,species,weight,years_old
0,"smith, fido",dog,5 lbs,1
1,"SMITH,, Bella",CAT,7 lbs,10
2,"Ortiz, Milo",dog,15 lbs,5
3,"Smith, Goldie",goldfish,7 oz,???
4,"Kim, Fuzz",cat,3 lbs,5 (five)
5,"Jones, Las Vegas",DOG,12 lbs,unknown
6,Jones; fifi,puppy,2 lbs,0.25
7,"Lee, nemo",gold fish,10 oz,2
8,"Chan, ZIGGY",kitten,2 lbs,0.5
9,"Chan, Zog",dog,20 lbs,7


We're going to clean the data so that you can __programmatically__ answer the following questions:

- Which family owns the most pets? (All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)
---
- How many of each kind of animal are there? (Puppies should count as dogs, and kittens as cats.)
---
- What is the total weight of the pets? (And which family's pets weigh the most, total?)
---
- Of the *known* ages, what's the average pet age?

## 1) Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

... normalized so that `.value_counts()` will count `smith, fido` and `SMITH, Bella` as being from the same family.

In [10]:
pets['last_name'] = pets['name'].str.extract(r'(\w+\W)')

In [13]:
pets['last_name'] = pets['last_name'].str.replace('(\W)','',regex=True)

In [15]:
pets['last_name'] = pets['last_name'].str.lower()

## 2) Check how many distinct last names you see

(There should be 6.)

In [20]:
len(pets.last_name.value_counts())
#there are 6

6

## 3) Calculate which family owns the most pets

(All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)

In [23]:
pets.last_name.value_counts().head(1)
#the smith family owns the most pets (3)

last_name
smith    3
Name: count, dtype: int64

## 4) Create a new column called `animal_type`, which standardizes the `species` column

(Puppies should count as dogs, and kittens as cats; and there should only be one spelling of `gold fish`/`goldfish`.)

In [31]:
pets['animal_type'] = (pets['species']
                       .str.lower()
                       .replace('puppy', 'dog', regex = True)
                       .replace('\s', '', regex = True)
                      )

## 5) How many of each kind of animal are there?

In [33]:
pets.animal_type.value_counts()

animal_type
dog         5
cat         2
goldfish    2
kitten      1
Name: count, dtype: int64

## 6) Create a new column called `weight_quantity`, which represents the numeric component of the `weight` column, as an integer

(i.e., `"10 oz"` -> `10`)

In [34]:
pets['weight_quantity'] = (pets['weight']
                           .str.extract(r'(\d+)')
                          )

## 7) Create a new column called `weight_unit`, which represents the units component of the `weight` column

(i.e., `"10 oz"` -> `"oz"`)

In [36]:
pets['weight_unit'] = (pets['weight']
                      .str.extract(r'([a-z]+)')
                      )

## 8) Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`

(i.e., `"10 oz"` -> `"oz"`)

You could do this by defining a function and using `column.apply(my_function)` or through Pandas' `.replace({ ... })` method, which we didn't cover in class but is somewhat easier here. That would look like this (yes, I'm giving you the answer here):

```python
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)
```

In [39]:
pets['unit_factor'] = (pets['weight_unit']
                      .replace({
                          'lbs':1, 
                          'oz': 0.0625
                      })
                      )

## 8) Create a new column called `weight_lbs`, which calculates the pet's weight in lbs by multiplying `weight_quantity` by `unit_factor`

In [46]:
pets= pets.astype({'weight_quantity': 'float', 'unit_factor': 'float'})

In [48]:
pets['weight_lbs'] = pets['weight_quantity'].mul(pets['unit_factor'])

## 9) What is the total weight of the pets?

In [52]:
pets.weight_lbs.sum()
#the total weight of the pets is 67.0625


,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor,weight_lbs
0,"smith, fido",dog,5 lbs,1,smith,dog,5.0,lbs,1.0000,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,smith,cat,7.0,lbs,1.0000,7.0000
2,"Ortiz, Milo",dog,15 lbs,5,ortiz,dog,15.0,lbs,1.0000,15.0000
3,"Smith, Goldie",goldfish,7 oz,???,smith,goldfish,7.0,oz,0.0625,0.4375
4,"Kim, Fuzz",cat,3 lbs,5 (five),kim,cat,3.0,lbs,1.0000,3.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,jones,dog,12.0,lbs,1.0000,12.0000
6,Jones; fifi,puppy,2 lbs,0.25,jones,dog,2.0,lbs,1.0000,2.0000
7,"Lee, nemo",gold fish,10 oz,2,lee,goldfish,10.0,oz,0.0625,0.6250
8,"Chan, ZIGGY",kitten,2 lbs,0.5,chan,kitten,2.0,lbs,1.0000,2.0000
9,"Chan, Zog",dog,20 lbs,7,chan,dog,20.0,lbs,1.0000,20.0000


## 10)  And which family's pets weigh the most, total?

In [51]:
pets.groupby('last_name')['weight_lbs'].sum()
#the chan family's pets weigh the most at 22 pounds

last_name
chan     22.0000
jones    14.0000
kim       3.0000
lee       0.6250
ortiz    15.0000
smith    12.4375
Name: weight_lbs, dtype: float64

## 11) Of the known ages, what's the average pet age?

(You should get `3.84375`; if not, make sure you're accounting for numbers like `0.5` correctly.)

In [75]:
pets['age'] = (pets['years_old']
               .str.replace('[a-z]+|\?|\(|\)', '', regex = True)
               .replace('', np.nan)
               .astype(float)
              )

In [76]:
pets['age'].mean()
#average pet age is 3.84375

3.84375

---

---

---